In [1]:
import cv2 as cv
import os
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
import datetime as dt

In [2]:
# Path for exported data, numpy arrays
DATA_PATH = os.path.join("D:\mediapipe_projects\hand_gesture_medium\data")

# Actions that we try to detect
actions = np.array(["hello", "thanks", "iloveyou", "no", "yes"])

# Thirty videos worth of data
no_sequences = 30

# Videos are going to be 30 frames in length
sequence_length = 30

for action in actions:
    for sequence in range(no_sequences):
        try:
            os.makedirs(os.path.join(DATA_PATH, action, str(sequence)))
        except:
            pass

In [3]:
actions = np.array(["hello", "thanks", "iloveyou", "no", "yes"])

In [7]:
label_map = {label:num for num, label in enumerate(actions)}
label_map

{'hello': 0, 'thanks': 1, 'iloveyou': 2, 'no': 3, 'yes': 4}

In [9]:
sequences, labels = [], []
for action in actions:
    for sequence in range(no_sequences):
        window = []
        for frame_num in range(sequence_length):
            res = np.load(os.path.join(DATA_PATH, action, str(sequence), "{}.npy".format(frame_num)))
            window.append(res)
        sequences.append(window)
        labels.append(label_map[action])

In [11]:
sequences

[[array([ 0.4575986 ,  0.47392583, -0.76169509, ...,  0.        ,
          0.        ,  0.        ]),
  array([ 0.46105701,  0.48192579, -1.00679421, ...,  0.        ,
          0.        ,  0.        ]),
  array([ 0.46471509,  0.48797625, -0.97652292, ...,  0.        ,
          0.        ,  0.        ]),
  array([ 0.47089434,  0.48793101, -0.92455637, ...,  0.        ,
          0.        ,  0.        ]),
  array([ 0.47663298,  0.4904418 , -0.90548766, ...,  0.        ,
          0.        ,  0.        ]),
  array([ 0.48227468,  0.49095386, -0.89895689, ...,  0.        ,
          0.        ,  0.        ]),
  array([ 0.48238063,  0.49013451, -0.91874027, ...,  0.        ,
          0.        ,  0.        ]),
  array([ 0.48172814,  0.49004471, -1.04417181, ...,  0.        ,
          0.        ,  0.        ]),
  array([ 0.48155046,  0.48936048, -0.98651171, ...,  0.        ,
          0.        ,  0.        ]),
  array([ 0.48138136,  0.48780674, -0.94817698, ...,  0.        ,
       

In [11]:
X = np.array(sequences)

In [13]:
X.shape

(150, 30, 1662)

In [15]:
np.array(labels).shape

(150,)

In [13]:
y = to_categorical(labels).astype(int)

In [15]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.05, shuffle=True, random_state=1)

In [17]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.callbacks import TensorBoard
from keras.callbacks import EarlyStopping

In [19]:
early_stopping = EarlyStopping(restore_best_weights=True, patience=10)

In [21]:
model = Sequential()
model.add(LSTM(64, return_sequences=True, activation="relu", input_shape=(30, 1662)))

model.add(LSTM(128, return_sequences=True, activation="relu"))

model.add(LSTM(64, return_sequences=False, activation="relu"))

model.add(Dense(64, activation="relu"))


model.add(Dense(32, activation="relu"))

model.add(Dense(actions.shape[0], activation="softmax"))

In [23]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 30, 64)            442112    
                                                                 
 lstm_1 (LSTM)               (None, 30, 128)           98816     
                                                                 
 lstm_2 (LSTM)               (None, 64)                49408     
                                                                 
 dense (Dense)               (None, 64)                4160      
                                                                 
 dense_1 (Dense)             (None, 32)                2080      
                                                                 
 dense_2 (Dense)             (None, 5)                 165       
                                                                 
Total params: 596741 (2.28 MB)
Trainable params: 596741 

In [25]:
model.compile(optimizer="Adam", loss="categorical_crossentropy", metrics=["categorical_accuracy"])

In [29]:
log_dir = os.path.join("Logs")
tb_callbacks = TensorBoard(log_dir=log_dir)

In [31]:
model.fit(X_train, y_train, epochs=175, callbacks=[tb_callbacks])

Epoch 1/175
5/5 [==============================] - 7s 128ms/step - loss: 2.4005 - categorical_accuracy: 0.1831
Epoch 2/175
5/5 [==============================] - 1s 122ms/step - loss: 2.2434 - categorical_accuracy: 0.1761
Epoch 3/175
5/5 [==============================] - 1s 123ms/step - loss: 1.8818 - categorical_accuracy: 0.2183
Epoch 4/175
5/5 [==============================] - 1s 123ms/step - loss: 1.6559 - categorical_accuracy: 0.1761
Epoch 5/175
5/5 [==============================] - 1s 131ms/step - loss: 1.5790 - categorical_accuracy: 0.2958
Epoch 6/175
5/5 [==============================] - 1s 131ms/step - loss: 1.6503 - categorical_accuracy: 0.3169
Epoch 7/175
5/5 [==============================] - 1s 122ms/step - loss: 1.5669 - categorical_accuracy: 0.3380
Epoch 8/175
5/5 [==============================] - 1s 127ms/step - loss: 1.4977 - categorical_accuracy: 0.3380
Epoch 9/175
5/5 [==============================] - 1s 121ms/step - loss: 1.3588 - categorical_accuracy: 0.4225
E

In [111]:
model_eval_loss, model_eval_acc = model.evaluate(X_test, y_test)
date_time_format = '%Y_%m_%d__%H_%M_%S'
current_date_time_dt = dt.datetime.now()
current_date_time_str = dt.datetime.strftime(current_date_time_dt, date_time_format)

model_name = f'model__date_time_{current_date_time_str}__loss_{model_eval_loss}__acc_{model_eval_acc}__hand.h5'
model.save(model_name)

1/1 [==============================] - 1s 650ms/step - loss: 0.6741 - categorical_accuracy: 0.7500


C:\Users\atif\anaconda3\Lib\site-packages\keras\src\engine\training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [59]:
# model.load_weights("D:/mediapipe_projects/hand_gesture_medium/models/model__date_time_2024_11_30__18_42_36__loss_8.536780660506338e-05__acc_1.0__hand.h5")

In [61]:
# from sklearn.metrics import multilabel_confusion_matrix, accuracy_score

In [63]:
# yhat = model.predict(X_test)

1/1 [==============================] - 1s 846ms/step


In [65]:
# ytrue = np.argmax(y_test, axis=1).tolist()
# yhat = np.argmax(yhat, axis=1).tolist()

In [67]:
# multilabel_confusion_matrix(ytrue, yhat)

array([[[2, 0],
        [0, 3]],

       [[3, 0],
        [0, 2]]], dtype=int64)

In [71]:
# accuracy_score(ytrue, yhat)

1.0

In [73]:
# import joblib

In [77]:
# date_time_format = '%Y_%m_%d__%H_%M_%S'
# current_date_time_dt = dt.datetime.now()
# current_date_time_str = dt.datetime.strftime(current_date_time_dt, date_time_format)

In [79]:
# model_name = f'model__date_time_{current_date_time_str}__loss_{model_eval_loss}__acc_{model_eval_acc}__hand.pkl'
# joblib.dump(model, model_name)

['model__date_time_2024_12_01__15_42_55__loss_1.1307454109191895__acc_0.0__hand.pkl']